In [1]:
import tensorflow as tf
from tensorflow.keras import layers, Model
import os

In [2]:
cd ./drive/My\ Drive/Colab\ Notebooks

/content/drive/My Drive/Colab Notebooks


In [3]:
folder = './aclImdb'

In [4]:
xtr = []
xte = []

ytr = []
yte = []
for dir in os.listdir(folder):
  if dir == 'test':
    for filedir in os.listdir(folder+'/'+dir):
      filefolder = folder+'/'+dir+'/'+filedir

      if filedir == 'neg':
        for file in os.listdir(filefolder):
          filepath = os.path.join(filefolder, file)
          xte.append(open(filepath, encoding='UTF-8').read().strip())
          yte.append(0)
      if filedir == 'pos':
        for file in os.listdir(filefolder):
          filepath = os.path.join(filefolder, file)
          xte.append(open(filepath, encoding='UTF-8').read().strip())
          yte.append(1)
    # print('test')
  if dir == 'train':
    for filedir in os.listdir(folder+'/'+dir):
      filefolder = folder+'/'+dir+'/'+filedir

      if filedir == 'neg':
        for file in os.listdir(filefolder):
          filepath = os.path.join(filefolder, file)
          xtr.append(open(filepath, encoding='UTF-8').read().strip())
          ytr.append(0)
      if filedir == 'pos':
        for file in os.listdir(filefolder):
          filepath = os.path.join(filefolder, file)
          xtr.append(open(filepath, encoding='UTF-8').read().strip())
          ytr.append(1)


In [5]:
print(len(xtr), len(xte))
print(len(ytr), len(yte))
print(xtr[1], ytr[1])
print(xte[1], yte[1])

25000 25000
25000 25000
"I Am Curious: Yellow" is a risible and pretentious steaming pile. It doesn't matter what one's political views are because this film can hardly be taken seriously on any level. As for the claim that frontal male nudity is an automatic NC-17, that isn't true. I've seen R-rated films with male nudity. Granted, they only offer some fleeting views, but where are the R-rated films with gaping vulvas and flapping labia? Nowhere, because they don't exist. The same goes for those crappy cable shows: schlongs swinging in the breeze but not a clitoris in sight. And those pretentious indie movies like The Brown Bunny, in which we're treated to the site of Vincent Gallo's throbbing johnson, but not a trace of pink visible on Chloe Sevigny. Before crying (or implying) "double-standard" in matters of nudity, the mentally obtuse should take into account one unavoidably obvious anatomical difference between men and women: there are no genitals on display when actresses appears

In [6]:
import re

def preprocess(w):
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
  w = w.strip()
  return w

In [7]:
xtrain = []
xtest = []
for s in xtr:
  xtrain.append(preprocess(s))
for s in xte:
  xtest.append(preprocess(s))

print(xtrain[1])
print(xtest[1])

I Am Curious Yellow is a risible and pretentious steaming pile . It doesn t matter what one s political views are because this film can hardly be taken seriously on any level . As for the claim that frontal male nudity is an automatic NC , that isn t true . I ve seen R rated films with male nudity . Granted , they only offer some fleeting views , but where are the R rated films with gaping vulvas and flapping labia ? Nowhere , because they don t exist . The same goes for those crappy cable shows schlongs swinging in the breeze but not a clitoris in sight . And those pretentious indie movies like The Brown Bunny , in which we re treated to the site of Vincent Gallo s throbbing johnson , but not a trace of pink visible on Chloe Sevigny . Before crying or implying double standard in matters of nudity , the mentally obtuse should take into account one unavoidably obvious anatomical difference between men and women there are no genitals on display when actresses appears nude , and the same 

In [8]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

In [9]:
train_tensor, inp_lang_tokenizer = tokenize(xtrain)

In [10]:
test_tensor = inp_lang_tokenizer.texts_to_sequences(xtest)
test_tensor = tf.keras.preprocessing.sequence.pad_sequences(test_tensor,
                                                        padding='post')

print(len(train_tensor), len(test_tensor))

25000 25000


In [11]:
from tensorflow.keras.utils import to_categorical

train_label, test_label = to_categorical(ytr), to_categorical(yte)
vocab_size = len(inp_lang_tokenizer.word_index)+1

In [12]:
model = tf.keras.Sequential([
        layers.Embedding(vocab_size, 64),
        layers.Bidirectional(layers.LSTM(64, return_sequences=True)),
        layers.Bidirectional(layers.LSTM(32)),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(2, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          4689792   
_________________________________________________________________
bidirectional (Bidirectional (None, None, 128)         66048     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 130       
Total params: 4,801,346
Trainable params: 4,801,346
Non-trainable params: 0
______________________________________________

In [13]:
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy', tf.keras.metrics.AUC()])

In [14]:
results = model.fit(train_tensor, train_label, epochs=10, batch_size=256, validation_split = 0.2)

Epoch 1/10
79/79 [==============================] - 58s 729ms/step - loss: 0.6695 - accuracy: 0.6238 - auc: 0.6277 - val_loss: 0.9813 - val_accuracy: 0.0000e+00 - val_auc: 0.0000e+00
Epoch 2/10
79/79 [==============================] - 57s 718ms/step - loss: 0.6582 - accuracy: 0.6250 - auc: 0.6474 - val_loss: 0.9967 - val_accuracy: 0.0000e+00 - val_auc: 0.0000e+00
Epoch 3/10
79/79 [==============================] - 57s 718ms/step - loss: 0.5719 - accuracy: 0.7138 - auc: 0.7872 - val_loss: 0.9690 - val_accuracy: 0.5238 - val_auc: 0.3705
Epoch 4/10
79/79 [==============================] - 57s 719ms/step - loss: 0.3740 - accuracy: 0.8564 - auc: 0.9176 - val_loss: 0.6103 - val_accuracy: 0.7608 - val_auc: 0.7499
Epoch 5/10
79/79 [==============================] - 57s 718ms/step - loss: 0.2406 - accuracy: 0.9168 - auc: 0.9622 - val_loss: 0.4561 - val_accuracy: 0.8332 - val_auc: 0.8690
Epoch 6/10
79/79 [==============================] - 57s 720ms/step - loss: 0.1833 - accuracy: 0.9427 - auc: 0

In [15]:
loss, acc, auc = model.evaluate(test_tensor, test_label)
print("test auc:", auc)

782/782 [==============================] - 123s 157ms/step - loss: 0.6058 - accuracy: 0.8338 - auc: 0.9009
test auc: 0.9008569717407227
